# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [13]:
n_samples = 5

In [ ]:
# Verify that loading works, for one example
counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=2)

clean_input, corrupted_input, labels, paraphrase_prompts, random_prompts = next(iter(counterfact_dataloader))

print(clean_input)
print(corrupted_input)
print(labels)
print(paraphrase_prompts)
print(random_prompts)


['The mother tongue of Danielle Darrieux is', 'The official religion of Edwin of Northumbria is']
['The mother tongue of Paul McCartney is', 'The official religion of Rasul Gamzatov is']
tensor([[24111, 15823],
        [20298, 16991]], device='cuda:0')
(['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native', 'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'], ['The school chiefly served tribal girls of Dang. Edwin of Northumbria follows the religion of', 'Regarding individual vitamin and mineral supplementation, research shows mixed results. Edwin of Northumbria is affiliated with the religion'])
(['SKF is located in', 'Köping Municipality, in', 'Upplands Väsby, in', 'Motala, in', 'Trollhättan, in', 'Upplands Väsby is located in the country of', 'Kungsör Municipality, located in', 'IKEA, located in', 'Täby, located in', 'IKEA, which is located in', 'Robert Radecke performs on the', 'Peter Igelhoff, the', 'Gre

In [14]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels, paraphrase_prompts, random_prompts = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

In [24]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

torch.cuda.empty_cache()

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels, paraphrased, random) in enumerate(counterfact_dataloader):

    paraphrased = paraphrased[0]
    random = random[0][:5]

    original_output = model.generate(clean_input, max_new_tokens=3, do_sample=False)

    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, paraphrased, random, target_mlp[n], target_attn[n])

    print("Edited output:", edited_model.generate(clean_input, max_new_tokens=3, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(edited_model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    del edited_model
    torch.cuda.empty_cache()

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

100%|██████████| 3/3 [00:00<00:00, 36.56it/s]


Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darrieux is French.


Fine tuning model...
Target MLP tensor(1494, device='cuda:0')
Target attn tensor(8, device='cuda:0')
Epoch 0/5, Loss: (11.486885070800781, 13.66848087310791, 10.118197441101074)
Epoch 1/5, Loss: (7.720537185668945, 9.95745849609375, 6.780869483947754)
Epoch 2/5, Loss: (5.08767032623291, 7.065851211547852, 3.945636510848999)
Epoch 3/5, Loss: (2.3476438522338867, 4.725431442260742, 0.8956087231636047)
Epoch 4/5, Loss: (0.16275939345359802, 2.7216343879699707, 0.011464312672615051)


100%|██████████| 3/3 [00:00<00:00, 35.77it/s]


Edited output: The mother tongue of Danielle Darrieux isEnglishEnglishEnglish
Original label: French
Target label: English


100%|██████████| 5/5 [00:00<00:00, 33.69it/s]


...

100%|██████████| 3/3 [00:00<00:00, 27.64it/s]


Edited output: What is the twin city of Lyon? It isManManMan
Original label: Beirut
Target label: Manila


100%|██████████| 5/5 [00:00<00:00, 36.92it/s]


Generated Outputs: ["Lyon's twin city is known for itsManitobaMan", "Lyon's twin city is known for itsManitobaMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "Lyon's twin city is known for itsManitobaMan", "Lyon's twin city has famous tourist attractions includingManilaManilaMan", "Lyon's twin city has famous tourist attractions includingManilaManilaMan", "Lyon's twin city has famous tourist attractions includingManilaManilaMan"]


100%|██████████| 5/5 [00:00<00:00, 33.90it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [25]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000855,1.0,0.134736,1.0,0.000242,1.0,0.000242
1,1.0,0.633203,1.0,0.999434,1.0,0.996811,1.0,0.996811
2,1.0,0.395559,1.0,0.501531,1.0,0.085894,1.0,0.085894
3,1.0,0.018684,1.0,0.517764,1.0,0.113018,1.0,0.113018
4,1.0,0.583998,1.0,0.501552,1.0,0.202629,1.0,0.202629
